In [1]:
from general_trader import GeneralTrader
import polars as pl
import polars_talib as pltalib
from trading_helpers import TradingData

In [2]:
traderData = TradingData(
    "/mnt/jonarne/dev/trading_data/",
    "/SP500/",
    "SP500_list.parquet",
)
# download_all_tickers. This function is only needed to call once,
# or when you decide to download new fresh ticker and sector data
#traderData.download_all_tickers()


# populate polars dataframe with open, close, high, low and volume data
ochl_data = traderData.load_ticker_parquet_files()
print(ochl_data.collect())


TradingData Initialized
shape: (4_300_704, 7)
┌─────────────────────┬───────────┬────────────┬───────────┬───────────┬─────────┬────────┐
│ date                ┆ close     ┆ high       ┆ low       ┆ open      ┆ volume  ┆ ticker │
│ ---                 ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---     ┆ ---    │
│ datetime[ns]        ┆ f64       ┆ f64        ┆ f64       ┆ f64       ┆ i64     ┆ str    │
╞═════════════════════╪═══════════╪════════════╪═══════════╪═══════════╪═════════╪════════╡
│ 1994-10-18 00:00:00 ┆ 1.205047  ┆ 1.214462   ┆ 1.186218  ┆ 1.205047  ┆ 418166  ┆ O      │
│ 1994-10-19 00:00:00 ┆ 1.308607  ┆ 1.318021   ┆ 1.214463  ┆ 1.214463  ┆ 249950  ┆ O      │
│ 1994-10-20 00:00:00 ┆ 1.318019  ┆ 1.365091   ┆ 1.318019  ┆ 1.327434  ┆ 141590  ┆ O      │
│ 1994-10-21 00:00:00 ┆ 1.299191  ┆ 1.327434   ┆ 1.289776  ┆ 1.318019  ┆ 68731   ┆ O      │
│ 1994-10-24 00:00:00 ┆ 1.270948  ┆ 1.327435   ┆ 1.270948  ┆ 1.299192  ┆ 75749   ┆ O      │
│ …                   ┆ …         

In [3]:
from datetime import datetime

    # Define a sample Polars expression

    # --- Test Cases ---

print("--- Testing Valid Instantiation ---")
try:
    trader = GeneralTrader(df_ohlc=ochl_data.collect())
    print("Successfully created GeneralTrader instance.")
    print("DataFrame Schema:", trader.df_ohlc.schema)
except ValidationError as e:
    print("Validation Error (unexpected):", e)
trader.df_ohlc

--- Testing Valid Instantiation ---
Successfully created GeneralTrader instance.
DataFrame Schema: Schema([('date', Datetime(time_unit='ns', time_zone=None)), ('close', Float64), ('high', Float64), ('low', Float64), ('open', Float64), ('volume', Int64), ('ticker', String)])


date,close,high,low,open,volume,ticker
datetime[ns],f64,f64,f64,f64,i64,str
1994-10-18 00:00:00,1.205047,1.214462,1.186218,1.205047,418166,"""O"""
1994-10-19 00:00:00,1.308607,1.318021,1.214463,1.214463,249950,"""O"""
1994-10-20 00:00:00,1.318019,1.365091,1.318019,1.327434,141590,"""O"""
1994-10-21 00:00:00,1.299191,1.327434,1.289776,1.318019,68731,"""O"""
1994-10-24 00:00:00,1.270948,1.327435,1.270948,1.299192,75749,"""O"""
…,…,…,…,…,…,…
2025-04-11 00:00:00,94.470001,95.480003,91.440002,93.349998,1634900,"""NRG"""
2025-04-14 00:00:00,95.919998,97.610001,95.0,97.449997,1981700,"""NRG"""
2025-04-15 00:00:00,97.110001,98.5,96.0,96.0,1513700,"""NRG"""


In [4]:

my_filter = pl.col("date").is_between(
    datetime(2019, 5, 22), datetime(2024, 5, 22), closed="none"
)
trader.add_pre_filter(my_filter)
trader.df_ohlc

date,close,high,low,open,volume,ticker
datetime[ns],f64,f64,f64,f64,i64,str
2019-05-23 00:00:00,51.660919,51.675557,50.819178,51.009485,2458327,"""O"""
2019-05-24 00:00:00,51.624371,51.968383,51.536536,51.653647,1347379,"""O"""
2019-05-28 00:00:00,51.046116,52.129397,51.046116,51.800023,2237892,"""O"""
2019-05-29 00:00:00,50.358086,51.075398,50.219018,51.01684,1676381,"""O"""
2019-05-30 00:00:00,50.665524,51.163247,50.314186,50.504493,1103414,"""O"""
…,…,…,…,…,…,…
2024-05-15 00:00:00,83.292854,84.781461,81.833829,81.833829,3448100,"""NRG"""
2024-05-16 00:00:00,81.754959,84.357552,81.607083,83.795629,3194100,"""NRG"""
2024-05-17 00:00:00,81.133888,83.10555,80.847996,81.774672,2913500,"""NRG"""


In [5]:
trader \
    .add_column("sma", pltalib.sma(timeperiod=200).over("ticker")) \
    .add_column("mean_B", pl.col("close").mean().over("ticker")) \
    .add_column("A_plus_B", pl.col("close") + pl.col("close"))


In [6]:

print(trader.added_columns)

{'sma': <Expr ['col("close")./home/user/dev/py…'] at 0x7FE65ADBFB50>, 'mean_B': <Expr ['col("close").mean().over([col(…'] at 0x7FE65AD92DD0>, 'A_plus_B': <Expr ['[(col("close")) + (col("close"…'] at 0x7FE65ADBFD90>}


In [8]:
trader.apply_added_columns()
trader.df_ohlc

date,close,high,low,open,volume,ticker,sma,mean_B,A_plus_B
datetime[ns],f64,f64,f64,f64,i64,str,f64,f64,f64
2019-05-23 00:00:00,51.660919,51.675557,50.819178,51.009485,2458327,"""O""",NaN,53.190879,103.321838
2019-05-24 00:00:00,51.624371,51.968383,51.536536,51.653647,1347379,"""O""",NaN,53.190879,103.248741
2019-05-28 00:00:00,51.046116,52.129397,51.046116,51.800023,2237892,"""O""",NaN,53.190879,102.092232
2019-05-29 00:00:00,50.358086,51.075398,50.219018,51.01684,1676381,"""O""",NaN,53.190879,100.716171
2019-05-30 00:00:00,50.665524,51.163247,50.314186,50.504493,1103414,"""O""",NaN,53.190879,101.331047
…,…,…,…,…,…,…,…,…,…
2024-05-15 00:00:00,83.292854,84.781461,81.833829,81.833829,3448100,"""NRG""",50.482437,35.900752,166.585709
2024-05-16 00:00:00,81.754959,84.357552,81.607083,83.795629,3194100,"""NRG""",50.708866,35.900752,163.509918
2024-05-17 00:00:00,81.133888,83.10555,80.847996,81.774672,2913500,"""NRG""",50.934359,35.900752,162.267776
